In [ ]:
import pandas as pd

# Load data
df = pd.read_csv('Input/Tweets.csv')

df_clean = df.dropna(subset=['tweet_location']).copy()


In [2]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm

In [3]:

geolocator = Nominatim(user_agent="tweet_geo_cleaner", timeout=10)
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1) 


def get_coordinates(location):
    try:
        location = location.strip()  
        if not location:
            return (None, None)
       
        geo = geocode(f"{location}, USA")
        if geo:
            return (geo.latitude, geo.longitude)
        return (None, None)
    except:
        return (None, None)


unique_locations = df_clean['tweet_location'].unique()
location_to_coord = {}
for loc in tqdm(unique_locations, desc="Geocoding"):
    location_to_coord[loc] = get_coordinates(loc)


df_clean['tweet_coord'] = df_clean['tweet_location'].map(location_to_coord)


df_clean = df_clean.dropna(subset=['tweet_coord'])
print(f"Final cleaned shape: {df_clean.shape}")

Geocoding: 100%|██████████| 3081/3081 [53:52<00:00,  1.05s/it]  

Final cleaned shape: (9907, 15)


In [4]:
df_clean.head(5)

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,"(None, None)",2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
6,570300616901320704,positive,0.6745,NaN,0.0,Virgin America,NaN,cjmcginnis,NaN,0,"@VirginAmerica yes, nearly every time I fly VX...","(37.7792588, -122.4193286)",2015-02-24 11:13:57 -0800,San Francisco CA,Pacific Time (US & Canada)
7,570300248553349120,neutral,0.6340,NaN,NaN,Virgin America,NaN,pilot,NaN,0,@VirginAmerica Really missed a prime opportuni...,"(34.0536909, -118.242766)",2015-02-24 11:12:29 -0800,Los Angeles,Pacific Time (US & Canada)
8,570299953286942721,positive,0.6559,NaN,NaN,Virgin America,NaN,dhepburn,NaN,0,"@virginamerica Well, I didn't…but NOW I DO! :-D","(32.7174202, -117.162772)",2015-02-24 11:11:19 -0800,San Diego,Pacific Time (US & Canada)
9,570295459631263746,positive,1.0000,NaN,NaN,Virgin America,NaN,YupitsTate,NaN,0,"@VirginAmerica it was amazing, and arrived an ...","(34.0536909, -118.242766)",2015-02-24 10:53:27 -0800,Los Angeles,Eastern Time (US & Canada)


2nd pre-processing

In [5]:
import requests
import pandas as pd
from dateutil import parser

df = df_clean.copy()

def parse_coordinates(coord):
    try:
        # coord = eval(coord)  # Convert string to tuple
        if isinstance(coord, (list, tuple)) and len(coord) == 2:
            return coord[0], coord[1]
    except:
        return None, None

In [6]:
df["latitude"], df["longitude"] = zip(*df["tweet_coord"].apply(parse_coordinates))

df = df.dropna(subset=["latitude", "longitude"])

df["latitude"] = df["latitude"].round(2)
df["longitude"] = df["longitude"].round(2)


df["tweet_created"] = df["tweet_created"].apply(lambda x: parser.parse(x[:-6]))  # Remove timezone offset

df["date"] = df["tweet_created"].dt.strftime("%Y-%m-%d")


weather_cache = {}

In [7]:

def get_weather_open_meteo(lat, lon, date):
    """Fetches historical weather for a given location & date using Open-Meteo API"""

    if pd.isna(lat) or pd.isna(lon):
        return None, None, None, None, None

    cache_key = (lat, lon, date) 

    if cache_key in weather_cache:
        return weather_cache[cache_key]  

    url = f"https://archive-api.open-meteo.com/v1/archive?latitude={lat}&longitude={lon}&start_date={date}&end_date={date}&hourly=temperature_2m,relativehumidity_2m,precipitation,windspeed_10m,snowfall"

    try:
        response = requests.get(url).json()
        if "hourly" in response:
            # Extract the first hour's data (since we're querying for a single day)
            temp = response["hourly"]["temperature_2m"][0]
            humidity = response["hourly"]["relativehumidity_2m"][0]
            precipitation = response["hourly"]["precipitation"][0]
            wind_speed = response["hourly"]["windspeed_10m"][0]
            snowfall = response["hourly"]["snowfall"][0]

            # Store in cache
            weather_cache[cache_key] = (temp, humidity, precipitation, wind_speed, snowfall)
            return temp, humidity, precipitation, wind_speed, snowfall
    except Exception as e:
        print(f"Error fetching data for {lat}, {lon} on {date}: {e}")
        return None, None, None, None, None

    return None, None, None, None, None

In [8]:
weather_data = df.apply(
    lambda row: get_weather_open_meteo(row["latitude"], row["longitude"], row["date"]), axis=1
)

In [9]:
df.tail()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,latitude,longitude,date
14631,569588464896876545,negative,1.0000,Bad Flight,1.0000,American,NaN,MDDavis7,NaN,0,@AmericanAir thx for nothing on getting us out...,"(39.7837304, -100.445882)",2015-02-22 12:04:07,US,Eastern Time (US & Canada),39.78,-100.45,2015-02-22
14633,569587705937600512,negative,1.0000,Cancelled Flight,1.0000,American,NaN,RussellsWriting,NaN,0,@AmericanAir my flight was Cancelled Flightled...,"(34.0536909, -118.242766)",2015-02-22 12:01:06,Los Angeles,Arizona,34.05,-118.24,2015-02-22
14636,569587371693355008,negative,1.0000,Customer Service Issue,1.0000,American,NaN,itsropes,NaN,0,@AmericanAir leaving over 20 minutes Late Flig...,"(31.2638905, -98.5456116)",2015-02-22 11:59:46,Texas,NaN,31.26,-98.55,2015-02-22
14638,569587188687634433,negative,1.0000,Customer Service Issue,0.6659,American,NaN,SraJackson,NaN,0,"@AmericanAir you have my money, you change my ...","(40.0757384, -74.4041622)",2015-02-22 11:59:02,New Jersey,Eastern Time (US & Canada),40.08,-74.40,2015-02-22
14639,569587140490866689,neutral,0.6771,NaN,0.0000,American,NaN,daviddtwu,NaN,0,@AmericanAir we have 8 ppl so we need 2 know h...,"(32.7762719, -96.7968559)",2015-02-22 11:58:51,"dallas, TX",NaN,32.78,-96.80,2015-02-22


In [10]:
weather_df = pd.DataFrame(weather_data.tolist(), columns=["temperature", "humidity", "precipitation", "wind_speed", "snowfall"])


df = df.join(weather_df)

In [ ]:
df.to_csv('Input/df_pre-processed.csv', index=False)

In [12]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,...,tweet_location,user_timezone,latitude,longitude,date,temperature,humidity,precipitation,wind_speed,snowfall
6,570300616901320704,positive,0.6745,NaN,0.0,Virgin America,NaN,cjmcginnis,NaN,0,...,San Francisco CA,Pacific Time (US & Canada),37.78,-122.42,2015-02-24,16.2,34.0,0.0,12.2,0.0
7,570300248553349120,neutral,0.6340,NaN,NaN,Virgin America,NaN,pilot,NaN,0,...,Los Angeles,Pacific Time (US & Canada),34.05,-118.24,2015-02-24,15.5,40.0,0.0,13.9,0.0
8,570299953286942721,positive,0.6559,NaN,NaN,Virgin America,NaN,dhepburn,NaN,0,...,San Diego,Pacific Time (US & Canada),32.72,-117.16,2015-02-24,13.5,71.0,0.0,10.1,0.0
9,570295459631263746,positive,1.0000,NaN,NaN,Virgin America,NaN,YupitsTate,NaN,0,...,Los Angeles,Eastern Time (US & Canada),34.05,-118.24,2015-02-24,-12.3,56.0,0.0,15.1,0.0
11,570289724453216256,positive,1.0000,NaN,NaN,Virgin America,NaN,HyperCamiLax,NaN,0,...,NYC,America/New_York,40.71,-74.01,2015-02-24,-12.1,56.0,0.0,20.7,0.0
